In [27]:
from pyscf import gto, scf
import numpy as np
import scipy as sp
import sys
sys.path.append('/workspaces/Huckel-on-benzene/project/Benzene/rhf')
import plottingsystem as pts

In [40]:
benzene_molecule = '''
  C    1.2116068    0.6995215    0.0000000
  C    1.2116068   -0.6995215    0.0000000
  C   -0.0000000   -1.3990430   -0.0000000
  C   -1.2116068   -0.6995215   -0.0000000
  C   -1.2116068    0.6995215   -0.0000000
  C    0.0000000    1.3990430    0.0000000
  H    2.1489398    1.2406910    0.0000000
  H    2.1489398   -1.2406910    0.0000000
  H   -0.0000000   -2.4813820   -0.0000000
  H   -2.1489398   -1.2406910   -0.0000000
  H   -2.1489398    1.2406910   -0.0000000
  H    0.0000000    2.4813820    0.0000000
'''
extra_bound = [(0,5)]
coordinates = pts.taking_x_and_y_coordinate(pts.taking_carbon_coordinates(benzene_molecule))
print(coordinates)


[[ 1.2116068  0.6995215]
 [ 1.2116068 -0.6995215]
 [-0.        -1.399043 ]
 [-1.2116068 -0.6995215]
 [-1.2116068  0.6995215]
 [ 0.         1.399043 ]]


In [29]:
mol = gto.M(atom = benzene_molecule, basis ='sto-3g' )
print(len(mol.ao_labels()))
print(mol.ao_labels())
pz_indices = [i for i, label in enumerate(mol.ao_labels()) if 'C 2pz' in label]
print(pz_indices)


36
['0 C 1s    ', '0 C 2s    ', '0 C 2px   ', '0 C 2py   ', '0 C 2pz   ', '1 C 1s    ', '1 C 2s    ', '1 C 2px   ', '1 C 2py   ', '1 C 2pz   ', '2 C 1s    ', '2 C 2s    ', '2 C 2px   ', '2 C 2py   ', '2 C 2pz   ', '3 C 1s    ', '3 C 2s    ', '3 C 2px   ', '3 C 2py   ', '3 C 2pz   ', '4 C 1s    ', '4 C 2s    ', '4 C 2px   ', '4 C 2py   ', '4 C 2pz   ', '5 C 1s    ', '5 C 2s    ', '5 C 2px   ', '5 C 2py   ', '5 C 2pz   ', '6 H 1s    ', '7 H 1s    ', '8 H 1s    ', '9 H 1s    ', '10 H 1s    ', '11 H 1s    ']
[4, 9, 14, 19, 24, 29]


In [30]:
MAXITER = 2000
tol = 1.0e-6

In [31]:
S = mol.intor('int1e_ovlp')
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H_core = T + V
eri = mol.intor('int2e')


In [32]:
enuc = mol.get_enuc()
ndocc = mol.nelec[0]
print(enuc, ndocc)

203.01933448917916 21


In [33]:
# ==> Inspecting S for AO orthonormality <==
def controleer_orthonormaliteit(matrix):
    is_ortho = np.allclose(np.eye(len(matrix[0])), matrix)
    if is_ortho:
        print("De matrix is orthonormaal.")
    else:
        print("De matrix is niet orthonormaal.")
controleer_orthonormaliteit(S)

De matrix is niet orthonormaal.


In [34]:
# ==> Construct AO orthogonalization matrix A <==
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)
A = np.asarray(A)
ASA = A@S@A

# Check orthonormality
controleer_orthonormaliteit(ASA)


De matrix is orthonormaal.


In [35]:
# ==> Compute C & D matrices with CORE guess <==
# Transformed Fock matrix
F_p = A @ H_core @ A.T

# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
epsilon, C_p = np.linalg.eigh(F_p)

# Transform C_p back into AO basis
C = A @ C_p
# Grab occupied orbitals
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals
D = np.einsum('ik, kj -> ij',C_occ, C_occ.T, optimize=True)
print(D)

[[ 1.05161681e+00 -2.63961502e-01  1.73851319e-01 ...  5.06437947e-04
   8.34438602e-04  2.39235416e-03]
 [-2.63961502e-01  1.62463133e+00 -1.05463266e+00 ... -3.95740403e-03
  -1.60496482e-03 -4.05175605e-02]
 [ 1.73851319e-01 -1.05463266e+00  1.22840846e+00 ...  2.26210092e-03
   6.30035157e-03  3.40628298e-02]
 ...
 [ 5.06437947e-04 -3.95740403e-03  2.26210092e-03 ...  1.24946462e-02
  -5.53348775e-04 -9.60171853e-04]
 [ 8.34438602e-04 -1.60496482e-03  6.30035157e-03 ... -5.53348775e-04
   1.24946462e-02 -5.53347750e-04]
 [ 2.39235416e-03 -4.05175605e-02  3.40628298e-02 ... -9.60171853e-04
  -5.53347750e-04  1.24946421e-02]]


In [36]:
def apply_lagrange_multipliers(F, constraints, lambdas):
    for mu in range(len(constraints)):
        C_mu = constraints[mu]
        F += lambdas[mu] * np.outer(C_mu, C_mu)
    return F

In [37]:
def normaliseer(eigenvectors):
    eigenvectors = eigenvectors.T
    for index, eigenvector in enumerate(eigenvectors):
        eigenvectors[index] = eigenvector * (np.dot(eigenvector, eigenvector))**(-1/2)
    return eigenvectors.T

In [38]:
# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):

    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', eri, D, optimize=True)
    K = np.einsum('prqs,rs->pq', eri, D, optimize=True)
    
    F = H_core + 2 * J - K
    # Compute RHF energy
    SCF_E = enuc + np.einsum('pq,pq->', (H_core + F), D, optimize=True)
    SCF_E = enuc + np.sum( (H_core + F)*D)
    
    
    
    # SCF Converged?
    if (abs(SCF_E - E_old) < tol):
        break
    E_old = SCF_E
    
    # Compute new orbital guess
    epsilon, C_p = np.linalg.eigh(A.T @ F @ A)
    C = A @ C_p
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,iq->pq', C_occ, C_occ.T, optimize=True)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        raise Exception("Maximum number of SCF iterations exceeded.")
# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))
print(C)



==> Starting SCF Iterations <==


SCF converged.
Final RHF Energy: -227.89028017 [Eh]
[[ 2.86517788e-01 -4.96267063e-01  4.05419837e-01 ... -8.30783000e-02
  -1.10809021e-09  5.99478132e-02]
 [ 8.40102994e-03 -1.45511431e-02  9.20589682e-03 ...  5.93574719e-01
   1.53108391e-08 -4.50337652e-01]
 [ 1.87982568e-03  5.14037353e-04  1.34640738e-03 ... -1.49401606e-01
   3.76586350e-01  5.83083614e-01]
 ...
 [ 1.90960392e-03 -3.30755622e-03 -2.82113348e-03 ...  2.70268551e-01
   2.35166454e-08  2.83576466e-01]
 [-1.90961895e-03 -3.30754754e-03 -2.82113348e-03 ...  2.70268462e-01
  -2.35166338e-08 -2.83576466e-01]
 [-3.81923614e-03  8.67391474e-09 -2.82114675e-03 ... -8.89209407e-08
  -1.27797078e-14  2.83576404e-01]]


In [39]:

# ==> SCF Iterations <==
# Pre-iteration energy declarations

constraints = [np.zeros(H_core.shape[0])]
constraints[0][4] = 1.0
lambdas = [0.1]
target_pop = 1.5

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):

    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', eri, D, optimize=True)
    K = np.einsum('prqs,rs->pq', eri, D, optimize=True)
    
    F = H_core + 2 * J - K
    F = apply_lagrange_multipliers(F, constraints, lambdas)
    # Compute RHF energy
    SCF_E = enuc + np.einsum('pq,pq->', (H_core + F), D, optimize=True)
    SCF_E = enuc + np.sum( D * (H_core + F))
    
    
    # SCF Converged?
    E_old = SCF_E

    # Compute new orbital guess
    epsilon, C_p = np.linalg.eigh(A @ F @ A)
    C = A @ C_p
    C_occ = C[:, :ndocc]  
    D = np.einsum('pi,iq->pq', C_occ, C_occ.T, optimize=True)

    D_2 = np.einsum('pi,iq->pq', normaliseer(C)[:, :ndocc], normaliseer(C)[:, :ndocc].T, optimize=True)
    current_pop = D_2[4][4]*2
    print(current_pop)
    if(target_pop - current_pop<0):
        lambda_adjustment = lambdas[0] + (target_pop - current_pop)
    else:
        lambda_adjustment = lambdas[0] - (target_pop - current_pop)
    lambdas[0] = lambda_adjustment
    if( abs(SCF_E - E_old) < tol and np.abs(target_pop - current_pop) < tol):
        print(f"SCF converged in {scf_iter} iterations.")
        break
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        raise Exception("Maximum number of SCF iterations exceeded.")
# Post iterations
print(sum(D_2.diagonal()*2))
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

==> Starting SCF Iterations <==

0.788965515887067
2.0301226768112475
2.005931254158598
1.9436114007836895
1.9142775509622607
1.8993826297275416
1.88983088105451
1.882888882122517
1.8775495102952717
1.8732755350206471
1.8697726973253181
1.866838521047758
1.8643434230006684
1.862191664414625
1.8603159113000098
1.8586645642402788
1.857198934041167
1.8558885504118012
1.8547095395015165
1.8536426557259122
1.8526723418669484
1.8517858005772667
1.8509724444224585
1.850223411381835
1.8495312363652658
1.8488895772253855
1.848293013133898
1.8477368797539553


1.8472171417477914
1.8467302887794617
1.8462732524933754
1.8458433383804576
1.8454381702055653
1.8450556439763137
1.8446938897983547
1.8443512399620468
1.844026202162238
1.8437174368744487
1.8434237381694403
1.8431440173587574
1.8428772889992027
1.8426226588639436
1.8423793135656414
1.8421465115708708
1.841923575392655
1.841709884783793
1.8415048707840602
1.8413080104983415
1.8411188225027533
1.840936862792055
1.8407617211951262
1.8405930181964136
1.8404304021105233
1.8402735465648692
1.8401221482517571
1.839975924916754
1.839834613554768
1.8396979687891615
1.8395657614125371
1.8394377770706176
1.8393138150730788
1.8391936873172248
1.8390772173121739
1.8389642392927417
1.8388545974135229
1.838748145014797
1.8386447439528937
1.8385442639884801
1.8384465822269929
1.8383515826060983
1.8382591554256067
1.8381691969157983
1.8380816088405258
1.8379962981318707
1.8379131765534513
1.8378321603897838
1.8377531701593905
1.8376761303495255
1.8376009691706663
1.8375276183290539
1.8374560128157515


Exception: Maximum number of SCF iterations exceeded.